In [1]:
!pip install transformers
!pip install torchsummary
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00


In [2]:
from transformers import ElectraModel, ElectraConfig, ElectraForMaskedLM, ElectraTokenizer, PretrainedConfig, ElectraForTokenClassification
import numpy as np
from torchsummary import summary
import torch
from torch import nn
from datasets import load_dataset
from torch.utils.data import DataLoader
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

# Some tests, you can directly go to next part

In [3]:
config = ElectraConfig()
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')


discriminator = ElectraModel.from_pretrained('google/electra-small-discriminator')
inputs = tokenizer("hello my dog is cute", return_tensors="pt")
#print(inputs)
outputs = discriminator(**inputs, )
last_hidden_states = outputs.last_hidden_state
#print(last_hidden_states, last_hidden_states.shape)

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
labels = tokenizer("The capital of France is Paris.", return_tensors="pt")
print(inputs, labels)
model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator')
outputs = model(**inputs, labels=labels["input_ids"])
loss = outputs.loss
logits = outputs.logits
print(torch.argmax(logits, dim = -1))
print(tokenizer.decode(torch.squeeze(torch.argmax(logits, dim = -1)).tolist()))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

{'input_ids': tensor([[ 101, 1996, 3007, 1997, 2605, 2003,  103, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])} {'input_ids': tensor([[ 101, 1996, 3007, 1997, 2605, 2003, 3000, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

tensor([[1012, 1996, 3007, 1997, 2605, 2003, 3000, 1012, 1012]])
. the capital of france is paris..


In [4]:
print(tokenizer.mask_token_id)

103


In [5]:
print(discriminator)

ElectraModel(
  (embeddings): ElectraEmbeddings(
    (word_embeddings): Embedding(30522, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
  (encoder): ElectraEncoder(
    (layer): ModuleList(
      (0-11): 12 x ElectraLayer(
        (attention): ElectraAttention(
          (self): ElectraSelfAttention(
            (query): Linear(in_features=256, out_features=256, bias=True)
            (key): Linear(in_features=256, out_features=256, bias=True)
            (value): Linear(in_features=256, out_features=256, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): ElectraSelfOutput(
            (dense): Linear(in_features=256, out_features=256, bias=True)
            (LayerNorm): LayerNorm((

In [6]:
"""generator_config = {
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": True,
  "transformers_version": "4.36.2",
  "type_vocab_size": 2,
  "use_cache": True,
  "vocab_size": 30522
}

discriminator_config = {
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": True,
  "transformers_version": "4.36.2",
  "type_vocab_size": 2,
  "use_cache": True,
  "vocab_size": 30522
}"""

'generator_config = {\n  "attention_probs_dropout_prob": 0.1,\n  "embedding_size": 128,\n  "hidden_act": "gelu",\n  "hidden_dropout_prob": 0.1,\n  "hidden_size": 256,\n  "initializer_range": 0.02,\n  "intermediate_size": 1024,\n  "layer_norm_eps": 1e-12,\n  "max_position_embeddings": 512,\n  "model_type": "electra",\n  "num_attention_heads": 4,\n  "num_hidden_layers": 12,\n  "pad_token_id": 0,\n  "position_embedding_type": "absolute",\n  "summary_activation": "gelu",\n  "summary_last_dropout": 0.1,\n  "summary_type": "first",\n  "summary_use_proj": True,\n  "transformers_version": "4.36.2",\n  "type_vocab_size": 2,\n  "use_cache": True,\n  "vocab_size": 30522\n}\n\ndiscriminator_config = {\n  "attention_probs_dropout_prob": 0.1,\n  "embedding_size": 128,\n  "hidden_act": "gelu",\n  "hidden_dropout_prob": 0.1,\n  "hidden_size": 256,\n  "initializer_range": 0.02,\n  "intermediate_size": 1024,\n  "layer_norm_eps": 1e-12,\n  "max_position_embeddings": 512,\n  "model_type": "electra",\n  "n

# Load dataset

In [7]:
#Load IMDB dataset

#Input = masked data
#label = real data

#input = prediction of generator
#output = binary output

#We will masked 20% of the input

#103 is the encoding for [MASK]

def masking(x , mask_token = 103):
    k = int(len(x)*1/10)
    ids = random.sample(range(1, len(x)-1), k)
    for i in ids:
        x[i] = mask_token
    return x, ids


tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
dataset = load_dataset("scikit-learn/imdb", split="train")

def preprocessing_fn(x, tokenizer):
    generator_labels = tokenizer.encode(
        x["review"],
        add_special_tokens=False,
        truncation=True,
        max_length=128,
        padding=False,
        return_attention_mask=False,
    )
    x["input_ids"], masked_ids = masking(generator_labels, 103)

    attention_mask = torch.ones((len(x["input_ids"])))
    x["labels"] = generator_labels
    x["attention_mask"] = attention_mask
    x["masked_ids"] = masked_ids
    return x

print(len(dataset))

n_samples = 2000  # the number of training example

# We first shuffle the data !
dataset = dataset.shuffle()

# Select n_samples
splitted_dataset = dataset.select(range(n_samples))

# Tokenize the dataset
splitted_dataset = splitted_dataset.map(
    preprocessing_fn, fn_kwargs={"tokenizer": tokenizer}
)


# Remove useless columns
splitted_dataset = splitted_dataset.select_columns(["input_ids", "attention_mask", "labels", "masked_ids"])

# Split the train and validation
splitted_dataset = splitted_dataset.train_test_split(test_size=0.2)

train_set = splitted_dataset["train"]
valid_set = splitted_dataset["test"]


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

50000


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [40]:
class DataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        max_label_length = max(len(feature["input_ids"]) for feature in batch)
        max_length_ids = max(len(feature["masked_ids"]) for feature in batch)
        for feature in batch:
            current_length = len(feature["input_ids"])
            current_length_mask = len(feature["masked_ids"])
            remainder = [tokenizer.pad_token_id] * (max_label_length - current_length)

            feature["input_ids"] = feature["input_ids"] + remainder
            feature["labels"] = feature["input_ids"][1:]
            feature["labels"] = [
                feature["labels"][i] if i < current_length - 1 else -100
                for i in range(max_label_length - 1)
            ]
            feature["input_ids"] = feature["input_ids"][:-1]
            feature["attention_mask"] = [
                1 if x < current_length else 0 for x in range(max_label_length - 1)
            ]
            feature["masked_ids"] = [
                feature["masked_ids"][i] if i < current_length_mask - 1 else -1
                for i in range(max_length_ids - 1)
            ]
        features = {
            "input_ids": torch.tensor([f["input_ids"] for f in batch]),
            "attention_mask": torch.tensor([f["attention_mask"] for f in batch]),
            "labels": torch.tensor([f["labels"] for f in batch]),
            "masked_ids": torch.tensor([f["masked_ids"] for f in batch]),
        }
        return features

print(train_set[0].keys())
print(len(train_set[0]["input_ids"]), len(train_set[0]["attention_mask"]), len(train_set[0]["labels"]))
print(len(train_set[1]["input_ids"]), len(train_set[1]["attention_mask"]), len(train_set[1]["labels"]))

data_collator = DataCollator(tokenizer)

batch_size = 32

train_dataloader = DataLoader(
    train_set, batch_size=batch_size, collate_fn=data_collator
)
valid_dataloader = DataLoader(
    valid_set, batch_size=batch_size, collate_fn=data_collator
)
n_valid = len(valid_set)
n_train = len(train_set)

batch = next(iter(train_dataloader))
print(batch["input_ids"].shape, batch["labels"].shape, batch["attention_mask"].shape)

dict_keys(['input_ids', 'attention_mask', 'labels', 'masked_ids'])
128 128 128
128 128 128
torch.Size([32, 127]) torch.Size([32, 127]) torch.Size([32, 127])


# Model instantiation

In [57]:
class GeneratorHead(nn.Module):
    """Prediction module for the generator, made up of two dense layers."""
    def __init__(self, config):
        super().__init__()

        self.LayerNorm = nn.LayerNorm(config.embedding_size, eps=config.layer_norm_eps)
        self.dense = nn.Linear(config.hidden_size, config.embedding_size)
        self.dense2 = nn.Linear(config.embedding_size, config.vocab_size)

    def forward(self, generator_hidden_states):
        hidden_states = self.dense(generator_hidden_states)
        hidden_states = torch.nn.GELU()(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        hidden_states = self.dense2(hidden_states)
        #hidden_states = torch.nn.Softmax(dim = -1)(hidden_states)

        return hidden_states

class Generator(nn.Module):
    """Complete generator module"""
    def __init__(self, generator_body, generator_head):
        super().__init__()
        self.generator_body = generator_body
        self.generator_head = generator_head

    def forward(self, input):
      output = self.generator_body(**input).last_hidden_state
      output = self.generator_head(output)
      return output

class DiscriminatorHead(nn.Module):
    """Discriminator module for the generator, made up of two dense layers."""
    def __init__(self, config):
        super().__init__()

        self.LayerNorm = nn.LayerNorm(config.embedding_size, eps=config.layer_norm_eps)
        self.dense = nn.Linear(config.hidden_size, config.embedding_size)
        self.dense2 = nn.Linear(config.embedding_size, 2)

    def forward(self, discriminator_hidden_states):
        hidden_states = self.dense(discriminator_hidden_states)
        hidden_states = torch.nn.GELU()(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        hidden_states = self.dense2(hidden_states)

        return hidden_states

class Discriminator(nn.Module):
    """Complete Discriminator"""
    def __init__(self, discriminator_body, discriminator_head):
        super().__init__()
        self.discriminator_body = discriminator_body
        self.discriminator_head = discriminator_head

    def forward(self, input):
      output = self.discriminator_body(**input).last_hidden_state
      output = self.discriminator_head(output)
      return output

class ELECTRALoss():
  def __init__(self, loss_weights=(1.0, 50.0)):
    self.loss_weights = loss_weights
    self.gen_loss_fc = nn.CrossEntropyLoss(ignore_index = -100)
    self.disc_loss_fc = nn.BCEWithLogitsLoss()

  def __call__(self, generator_pred, discriminator_pred, generator_labels, masked_ids):
    list_gen_pred = []
    list_gen_labels = []
    for i in range(masked_ids.shape[0]):
      for j in range(masked_ids.shape[1]):
        masked_index = masked_ids[i,j]
        list_gen_pred.append(torch.unsqueeze(generator_pred[i,masked_index,:],dim =0))
        list_gen_labels.append(torch.unsqueeze(generator_labels[i,masked_index], dim = 0))
    list_gen_pred = torch.concat(list_gen_pred, axis = 0)
    list_gen_labels = torch.concat(list_gen_labels, axis = 0)
    gen_loss = self.gen_loss_fc(list_gen_pred, list_gen_labels)
    gen_acc = torch.sum(torch.argmax(list_gen_pred, dim = -1) == list_gen_labels)/ len(list_gen_labels)*100

    generated_tokens = torch.argmax(generator_pred, dim = -1)

    discriminator_labels =torch.ones((discriminator_pred.shape[0], discriminator_pred.shape[1])).cuda()
    for i in range(masked_ids.shape[0]):
      for j in range(masked_ids.shape[1]):
        masked_index = masked_ids[i,j]
        if generated_tokens[i,masked_index] != generator_labels[i,masked_index]:
          discriminator_labels[i,masked_index] = 0
    discriminator_labels =  discriminator_labels.to(torch.int64).view(discriminator_labels.shape[0]*discriminator_labels.shape[1]).float()
    discriminator_pred = discriminator_pred.view(discriminator_pred.shape[0]*discriminator_pred.shape[1],discriminator_pred.shape[2])
    disc_loss = self.disc_loss_fc(discriminator_pred, discriminator_labels)
    a = torch.argmax(discriminator_pred, dim = -1)
    print(a.shape, discriminator_labels.shape)
    disc_acc = torch.sum(torch.argmax(discriminator_pred, dim = -1) == discriminator_labels)/ len(discriminator_labels)*100
    return gen_loss * self.loss_weights[0] + disc_loss * self.loss_weights[1], gen_acc, disc_acc

In [58]:
generator_config = ElectraConfig(vocab_size = tokenizer.vocab_size, embedding_size= 64, hidden_size = 128)
generator_body = ElectraModel(generator_config)
generator_head = GeneratorHead(generator_config)
generator = Generator(generator_body, generator_head)

discriminator_config = ElectraConfig(vocab_size = tokenizer.vocab_size)
discriminator_body = ElectraModel(discriminator_config)
discriminator_head = DiscriminatorHead(discriminator_config)
discriminator = Discriminator(discriminator_body,discriminator_head)

loss = ELECTRALoss()

# Training

In [59]:
from google.colab import drive
drive.mount('/content/drive')

# Create a folder
!mkdir -p "/content/drive/My Drive/LLM-projects-models"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
def training(generator, discriminator, n_epochs, train_dataloader, valid_dataloader, loss, tokenizer, lr=5e-5):

    #trained_gen = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').cuda()
    optimizer_gen = torch.optim.AdamW(
        generator.parameters(),
        lr=lr,
        eps=1e-08,
    )
    optimizer_disc = torch.optim.AdamW(
        discriminator.parameters(),
        lr=lr,
        eps=1e-08,
    )
    list_train_loss = []
    list_val_loss = []
    best_val_loss = 10**50
    generator.cuda()
    discriminator.cuda()
    for e in range(n_epochs):
        # ========== Training ==========

        # Set model to training mode
        generator.train()
        discriminator.train()

        # Tracking variables
        train_loss = 0
        for batch in tqdm(train_dataloader):
            batch = {k: v.cuda() for k, v in batch.items()}

            input_ids, attention_mask, labels, masked_ids =(
                batch["input_ids"],
                batch["attention_mask"],
                batch["labels"],
                batch["masked_ids"]
            )
            batch.pop("labels")
            batch.pop("masked_ids")
            optimizer_gen.zero_grad()
            optimizer_disc.zero_grad()
            # Forward pass
            generator_pred = generator(batch)
            #generator_pred = trained_gen(**batch).logits
            predicted_tokens = torch.argmax(generator_pred, dim = -1)
            batch_discriminator = batch

            input_ids_for_discriminator = labels
            for i in range(len(batch)):
              for id in masked_ids[i,:]:
                if id != -1:
                  input_ids_for_discriminator[i][id] = predicted_tokens[i][id]
            input_ids_for_discriminator[input_ids_for_discriminator == -100] = tokenizer.pad_token_id
            batch_discriminator["input_ids"] = input_ids_for_discriminator

            batch_discriminator = {k: v.cuda() for k, v in batch_discriminator.items()}
            discriminator_pred = discriminator(batch_discriminator)

            # Backward pass
            loss_value, train_gen_acc, train_disc_acc = loss(generator_pred, discriminator_pred,labels, masked_ids)
            loss_value.backward()
            optimizer_gen.step()
            optimizer_disc.step()
            train_loss += loss_value.detach().cpu().item()
        list_train_loss.append(train_loss / len(train_dataloader))

        # ========== Validation ==========
        if e%5 == 0:
          generator.eval()
          discriminator.eval()
          valid_loss = 0
          for batch in tqdm(valid_dataloader):
              batch = {k: v.cuda() for k, v in batch.items()}

              input_ids, attention_mask, labels, masked_ids =(
                  batch["input_ids"],
                  batch["attention_mask"],
                  batch["labels"],
                  batch["masked_ids"]
              )
              batch.pop("labels")
              batch.pop("masked_ids")

              # Forward pass
              generator_pred = generator(batch)
              predicted_tokens = torch.argmax(generator_pred, dim = -1)
              batch_discriminator = batch

              input_ids_for_discriminator = labels
              for i in range(len(batch)):
                for id in masked_ids[i,:]:
                  if id != -1:
                    input_ids_for_discriminator[i][id] = predicted_tokens[i][id]
              input_ids_for_discriminator[input_ids_for_discriminator == -100] = tokenizer.pad_token_id
              batch_discriminator["input_ids"] = input_ids_for_discriminator

              batch_discriminator = {k: v.cuda() for k, v in batch_discriminator.items()}
              discriminator_pred = discriminator(batch_discriminator)

              # Backward pass
              loss_value, val_gen_acc, val_disc_acc = loss(generator_pred, discriminator_pred,labels, masked_ids)

              valid_loss += loss_value.detach().cpu().item()
        list_val_loss.append(valid_loss / len(valid_dataloader))
        if list_val_loss[-1] < best_val_loss:
          best_val_loss = list_val_loss[-1]
          generator_path = f"/content/drive/My Drive/LLM-projects-models/generator_epoch{e}_lr{lr}"
          discriminator_path =  f"/content/drive/My Drive/LLM-projects-models/discriminator_epoch{e}_lr{lr}"
          torch.save(generator.state_dict(), generator_path)
          torch.save(discriminator.state_dict(), discriminator_path)


        print(
            e,
            "\n\t - Train loss: {:.4f}".format(list_train_loss[-1]),
            "\n\t - Val loss: {:.4f}".format(list_val_loss[-1]),
            "\n\t - Train gen acc: {:.4f}".format(train_gen_acc),
            "\n\t - Train disc acc: {:.4f}".format(train_disc_acc),
            "\n\t - Val gen acc: {:.4f}".format(val_gen_acc),
            "\n\t - Val disc acc: {:.4f}".format(val_disc_acc),
        )
    return list_train_loss, list_val_loss

In [61]:
list_train_loss, list_val_loss = training(generator, discriminator, 50, train_dataloader, valid_dataloader, loss, tokenizer, lr=5e-4)

  0%|          | 0/50 [00:00<?, ?it/s]


ValueError: Target size (torch.Size([4064])) must be the same as input size (torch.Size([4064, 2]))

In [46]:
plt.figure()
x_train = [i for i in range(len(list_train_loss))]
x_val = [i for i in range(len(list_val_loss))]
plt.plot(x_train, list_train_loss)
plt.plot(x_val, list_val_loss)

NameError: name 'list_train_loss' is not defined

<Figure size 640x480 with 0 Axes>

In [ ]:
# Example of target with class indice
loss = nn.CrossEntropyLoss(reduction = "mean")
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
print(input, target,output)

# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
print(input, target,output)